In [1]:
import pandas as pd
import numpy as np
import re
import os
import sys
import csv
import json
from datetime import datetime
from pathlib import Path

df = pd.read_excel('rawdata_loki.xlsx', sheet_name='Cleanser')

# Remove trailing .0 for all values (convert to str first)
df = df.applymap(lambda x: str(int(x)) if isinstance(x, float) and x.is_integer() else x)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 29 columns):
 #   Column                                                                                                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                  --------------  ----- 
 0   Response ID                                                                                                                                                                                             712 non-null    int64 
 1   Gender                                                                                                                                                                                                  712 no

C:\Users\aisar\AppData\Local\Temp\ipykernel_31040\3698390147.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(int(x)) if isinstance(x, float) and x.is_integer() else x)


In [2]:
df.columns = df.columns.str.strip()
df.columns

Index(['Response ID', 'Gender', 'Ethnicity', 'Age_range', 'State',
       'Avg_monthly_income',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Clarins.png" alt="Clarins" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Clinique1.png" alt="Clinique" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/EsteeLauder1.png" alt="Estee Lauder" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Kiehls1.png" alt="Kiehl's" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/LOccitane.png" alt="L'Occitane" style="width:200px;" />

In [3]:
df['Avg_monthly_income'] = df['Avg_monthly_income'].astype(str).replace({
    '1': 'Below RM2500',
    '2': 'RM2500 – RM4999',
    '3': 'RM2500 – RM4999',
    '4': 'RM2500 – RM4999',
    '5': 'RM5000 – RM6999',
    '6': 'RM5000 – RM6999',
    '7': 'RM7000 – RM10999',
    '8': 'RM7000 – RM10999',
    '9': 'RM7000 – RM10999',
    '10': 'RM7000 – RM10999',
    '11': 'RM7000 – RM10999'
})

df['State'] = df['State'].astype(str).replace({
    '1': 'WP',
    '2': 'Selangor'
})

df['Ethnicity'] = df['Ethnicity'].astype(str).replace({
    '1': 'Malay'
})

df['Gender'] = df['Gender'].astype(str).replace({
    '2': 'Female'
})

df['Age_range'] = df['Age_range'].astype(str).replace({
    '2': '26-35',
    '3': '26-35',
    '4': '36-45',
    '5': '36-45',
    '6': '46 and above',
    '7': '46 and above',
})


# df.drop(columns=['Please select the beer/stout brand that you purchase most often?'], inplace=True)

# df = df.rename(columns={
#     'Please select the diapers pants brand that you used most often.': 'Purchase Likelihood'
# })


# df = df.drop(df.columns[-3:], axis=1)

In [4]:
print(df['Age_range'].value_counts())
print('--------------')
print(df['Age_range'].isna().sum()) # No missing values

Age_range
26-35           495
36-45           193
46 and above     24
Name: count, dtype: int64
--------------
0


In [5]:
# Step 1: Replace string 'nan' with actual np.nan
df['Avg_monthly_income'] = df['Avg_monthly_income'].replace('nan', np.nan)

# Step 2: Sample values and impute
sampled_values = df['Avg_monthly_income'].dropna().sample(
    n=df['Avg_monthly_income'].isna().sum(),
    replace=True
)
df['Avg_monthly_income'] = df['Avg_monthly_income'].fillna(
    pd.Series(sampled_values.values, index=df[df['Avg_monthly_income'].isna()].index)
)

# Step 3: Check missing values again
print("Missing values after imputation:", df['Avg_monthly_income'].isna().sum())


Missing values after imputation: 0


In [6]:
columns = ['Gender', 'Ethnicity', 'State', 'Avg_monthly_income']

for col in columns:
    print(f"Column: {col}")
    print(df[col].value_counts(dropna=False))  # includes NaN counts in value_counts
    print(f"Missing values: {df[col].isna().sum()}")
    print("-" * 30)

Column: Gender
Gender
Female    712
Name: count, dtype: int64
Missing values: 0
------------------------------
Column: Ethnicity
Ethnicity
Malay    712
Name: count, dtype: int64
Missing values: 0
------------------------------
Column: State
State
WP          506
Selangor    206
Name: count, dtype: int64
Missing values: 0
------------------------------
Column: Avg_monthly_income
Avg_monthly_income
RM2500 – RM4999     481
RM5000 – RM6999     176
RM7000 – RM10999     55
Name: count, dtype: int64
Missing values: 0
------------------------------


In [7]:
df.columns

Index(['Response ID', 'Gender', 'Ethnicity', 'Age_range', 'State',
       'Avg_monthly_income',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Clarins.png" alt="Clarins" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Clinique1.png" alt="Clinique" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/EsteeLauder1.png" alt="Estee Lauder" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/Kiehls1.png" alt="Kiehl's" style="width:200px;" />:Which LUXURY SKIN CARE brands are you aware of for Cleanser?',
       '<img src="http://surveygizmolibrary.s3.amazonaws.com/library/642038/LOccitane.png" alt="L'Occitane" style="width:200px;" />

In [8]:
brand_mapping = {
    "Clarins": "Clarins Brand",
    "Clinique": "Clinique Brand",
    "Estee": "Estée Lauder Brand",
    "Kiehl's": "Kiehl’s Brand",
    "L'Occitane": "L’Occitane Brand",
    "Lancôme": "Lancôme Brand",
    "Laneige": "Laneige Brand",
    "Shiseido": "Shiseido Brand",
    "Innisfree": "Innisfree Brand",
    "SKII": "SK-II Brand",
    "Dior": "Dior Brand"
}

for keyword, new_name in brand_mapping.items():
    for col in df.columns:
        if keyword in col and "aware" in col:
            df = df.rename(columns={col: new_name})
            break  # stop once found


In [9]:
brand_mapping = {
    "Clarins": "Clarins Current",
    "Clinique": "Clinique Current",
    "Estee": "Estée Lauder Current",
    "Kiehl's": "Kiehl’s Current",
    "L'Occitane": "L’Occitane Current",
    "Lancôme": "Lancôme Current",
    "Laneige": "Laneige Current",
    "Shiseido": "Shiseido Current",
    "Innisfree": "Innisfree Current",
    "SKII": "SK-II Current",
    "Dior": "Dior Current"
}

for keyword, new_name in brand_mapping.items():
    for col in df.columns:
        if keyword in col and "currently" in col:
            df = df.rename(columns={col: new_name})
            break  # stop once found


In [10]:
df = df.rename(columns={
    'Which LUXURY SKIN CARE brands do you use most often for Cleanser.': 'Purchase Likelihood'
})

In [11]:
df['Purchase Likelihood'] = df['Purchase Likelihood'].astype(str).replace({
    '1': 'Clarins',
    '2': 'Clinique',
    '3': 'Estée Lauder',
    '4': "Kiehl’s",
    '5': "L’Occitane",
    '6': 'Lancôme',
    '7': 'Laneige',
    '8': 'Shiseido',
    '9': 'Innisfree',
    '10': 'SK-II',
    '11': 'Dior',
    '': np.nan,
    'nan': np.nan  # in case some rows got converted to string 'nan'
})

In [12]:
print(df['Purchase Likelihood'].value_counts())
print('--------------')
print(df['Purchase Likelihood'].isna().sum()) # No missing values

Purchase Likelihood
SK-II           119
Clinique         60
Clarins          47
Innisfree        46
Kiehl’s          41
Estée Lauder     25
Laneige          24
Lancôme          21
Shiseido         18
L’Occitane       11
Dior             10
Name: count, dtype: int64
--------------
290


In [13]:
# Get unique non-null brands
unique_brands = df['Purchase Likelihood'].dropna().unique()

# Create new binary columns based on each brand
for brand in unique_brands:
    col_name = f"{brand} Purchase Likelihood"
    df[col_name] = (df['Purchase Likelihood'] == brand).astype(int)

In [14]:
df.columns

Index(['Response ID', 'Gender', 'Ethnicity', 'Age_range', 'State',
       'Avg_monthly_income', 'Clarins Brand', 'Clinique Brand',
       'Estée Lauder Brand', 'Kiehl’s Brand', 'L’Occitane Brand',
       'Lancôme Brand', 'Laneige Brand', 'Shiseido Brand', 'Innisfree Brand',
       'SK-II Brand', 'Dior Brand', 'Clarins Current', 'Clinique Current',
       'Estée Lauder Current', 'Kiehl’s Current', 'L’Occitane Current',
       'Lancôme Current', 'Laneige Current', 'Shiseido Current',
       'Innisfree Current', 'SK-II Current', 'Dior Current',
       'Purchase Likelihood', 'SK-II Purchase Likelihood',
       'Kiehl’s Purchase Likelihood', 'Innisfree Purchase Likelihood',
       'Laneige Purchase Likelihood', 'Dior Purchase Likelihood',
       'Clarins Purchase Likelihood', 'Estée Lauder Purchase Likelihood',
       'Clinique Purchase Likelihood', 'L’Occitane Purchase Likelihood',
       'Lancôme Purchase Likelihood', 'Shiseido Purchase Likelihood'],
      dtype='object')

In [15]:
df = df.drop(columns=['Purchase Likelihood'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Response ID                       712 non-null    int64 
 1   Gender                            712 non-null    object
 2   Ethnicity                         712 non-null    object
 3   Age_range                         712 non-null    object
 4   State                             712 non-null    object
 5   Avg_monthly_income                712 non-null    object
 6   Clarins Brand                     216 non-null    object
 7   Clinique Brand                    277 non-null    object
 8   Estée Lauder Brand                242 non-null    object
 9   Kiehl’s Brand                     219 non-null    object
 10  L’Occitane Brand                  222 non-null    object
 11  Lancôme Brand                     247 non-null    object
 12  Laneige Brand         

In [16]:
print(df['Clarins Purchase Likelihood'].value_counts())
print('--------------')
print('missing values:', df['Clarins Purchase Likelihood'].isna().sum())

Clarins Purchase Likelihood
0    665
1     47
Name: count, dtype: int64
--------------
missing values: 0


### Pivoting

In [17]:
columns_to_process = df.columns.tolist()
columns_to_process

['Response ID',
 'Gender',
 'Ethnicity',
 'Age_range',
 'State',
 'Avg_monthly_income',
 'Clarins Brand',
 'Clinique Brand',
 'Estée Lauder Brand',
 'Kiehl’s Brand',
 'L’Occitane Brand',
 'Lancôme Brand',
 'Laneige Brand',
 'Shiseido Brand',
 'Innisfree Brand',
 'SK-II Brand',
 'Dior Brand',
 'Clarins Current',
 'Clinique Current',
 'Estée Lauder Current',
 'Kiehl’s Current',
 'L’Occitane Current',
 'Lancôme Current',
 'Laneige Current',
 'Shiseido Current',
 'Innisfree Current',
 'SK-II Current',
 'Dior Current',
 'SK-II Purchase Likelihood',
 'Kiehl’s Purchase Likelihood',
 'Innisfree Purchase Likelihood',
 'Laneige Purchase Likelihood',
 'Dior Purchase Likelihood',
 'Clarins Purchase Likelihood',
 'Estée Lauder Purchase Likelihood',
 'Clinique Purchase Likelihood',
 'L’Occitane Purchase Likelihood',
 'Lancôme Purchase Likelihood',
 'Shiseido Purchase Likelihood']

In [18]:
# Include Response ID for count
columns_to_process = ['Response ID', 'Gender', 'Age_range', 'Ethnicity','State','Avg_monthly_income'] + columns_to_process

# Melt the DataFrame for processing
melted_df = df.melt(id_vars=['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income'], value_vars=columns_to_process, 
                    var_name='Category', value_name='Value')

# Generate a pivot table without grand totals
pivot_table = pd.pivot_table(
    melted_df,
    values='Value',
    index=['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income'],
    columns='Category',
    aggfunc='count',
    fill_value=0
)

# Reset index for cleaner output (optional)
pivot_table.reset_index(inplace=True)
print(pivot_table)

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         26-35     Malay  Selangor    RM2500 – RM4999   
1         Female         26-35     Malay  Selangor    RM5000 – RM6999   
2         Female         26-35     Malay  Selangor   RM7000 – RM10999   
3         Female         26-35     Malay        WP    RM2500 – RM4999   
4         Female         26-35     Malay        WP    RM5000 – RM6999   
5         Female         26-35     Malay        WP   RM7000 – RM10999   
6         Female         36-45     Malay  Selangor    RM2500 – RM4999   
7         Female         36-45     Malay  Selangor    RM5000 – RM6999   
8         Female         36-45     Malay  Selangor   RM7000 – RM10999   
9         Female         36-45     Malay        WP    RM2500 – RM4999   
10        Female         36-45     Malay        WP    RM5000 – RM6999   
11        Female         36-45     Malay        WP   RM7000 – RM10999   
12        Female  46 and above     Malay  Selangor 

In [19]:
pivot_table.to_csv('checkcount_lokicleanser.csv')

In [20]:
total_rows = len(df)

# List of columns to process for current usage
awareness_columns = [col for col in df.columns if "Brand" in col]

# Loop through each column to calculate awareness and unawareness percentages
for column in awareness_columns:
    pivot_table[f'Unaware of {column}'] = pivot_table['Response ID'] - pivot_table[column]
    pivot_table[f'{column} Awareness (%)'] = (pivot_table[column] / total_rows) * 100
    pivot_table[f'{column} Unawareness (%)'] = (pivot_table[f'Unaware of {column}'] / total_rows) * 100

    # Round the percentages to 1 decimal place
    pivot_table[f'{column} Awareness (%)'] = pivot_table[f'{column} Awareness (%)']
    pivot_table[f'{column} Unawareness (%)'] = pivot_table[f'{column} Unawareness (%)']

# Drop the original columns after processing
pivot_table.drop(columns=awareness_columns, inplace=True)

# Drop columns starting with "Unaware of"
unaware_columns = [col for col in pivot_table.columns if col.startswith('Unaware of')]
pivot_table.drop(columns=unaware_columns, inplace=True)

# Define the columns for display, ensuring awareness and unawareness are side by side
display_columns1 = ['Gender', 'Age_range','Ethnicity','State','Avg_monthly_income']
for column in awareness_columns:
    display_columns1.extend([f'{column} Awareness (%)', f'{column} Unawareness (%)'])

print(pivot_table[display_columns1])

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         26-35     Malay  Selangor    RM2500 – RM4999   
1         Female         26-35     Malay  Selangor    RM5000 – RM6999   
2         Female         26-35     Malay  Selangor   RM7000 – RM10999   
3         Female         26-35     Malay        WP    RM2500 – RM4999   
4         Female         26-35     Malay        WP    RM5000 – RM6999   
5         Female         26-35     Malay        WP   RM7000 – RM10999   
6         Female         36-45     Malay  Selangor    RM2500 – RM4999   
7         Female         36-45     Malay  Selangor    RM5000 – RM6999   
8         Female         36-45     Malay  Selangor   RM7000 – RM10999   
9         Female         36-45     Malay        WP    RM2500 – RM4999   
10        Female         36-45     Malay        WP    RM5000 – RM6999   
11        Female         36-45     Malay        WP   RM7000 – RM10999   
12        Female  46 and above     Malay  Selangor 

In [21]:
total_rows = len(df)

# List of columns to process for current usage
usage_columns = [col for col in df.columns if "Current" in col]

# Loop through each column to calculate usage and non-usage percentages
for column in usage_columns:
    if column in pivot_table.columns:
        pivot_table[f'Not Using {column}'] = pivot_table['Response ID'] - pivot_table[column]
        pivot_table[f'{column} Usage (%)'] = (pivot_table[column] / total_rows) * 100
        pivot_table[f'{column} Non-usage (%)'] = (pivot_table[f'Not Using {column}'] / total_rows) * 100

        # Round the percentages to 1 decimal place
        pivot_table[f'{column} Usage (%)'] = pivot_table[f'{column} Usage (%)']
        pivot_table[f'{column} Non-usage (%)'] = pivot_table[f'{column} Non-usage (%)']
    else:
        print(f"Column '{column}' is missing in pivt_table.")

# Drop the original columns after processing
pivot_table.drop(columns=[col for col in usage_columns if col in pivot_table.columns], inplace=True)

# Drop columns starting with "Not Using"
not_using_columns = [col for col in pivot_table.columns if col.startswith('Not Using')]
pivot_table.drop(columns=not_using_columns, inplace=True)

# Define the columns for display, ensuring usage and non-usage are side by side
display_columns = ['Gender', 'Age_range', 'Ethnicity','State','Avg_monthly_income']
for column in usage_columns:
    if f'{column} Usage (%)' in pivot_table.columns and f'{column} Non-usage (%)' in pivot_table.columns:
        display_columns.extend([f'{column} Usage (%)', f'{column} Non-usage (%)'])

print(pivot_table[display_columns])

Category  Gender     Age_range Ethnicity     State Avg_monthly_income  \
0         Female         26-35     Malay  Selangor    RM2500 – RM4999   
1         Female         26-35     Malay  Selangor    RM5000 – RM6999   
2         Female         26-35     Malay  Selangor   RM7000 – RM10999   
3         Female         26-35     Malay        WP    RM2500 – RM4999   
4         Female         26-35     Malay        WP    RM5000 – RM6999   
5         Female         26-35     Malay        WP   RM7000 – RM10999   
6         Female         36-45     Malay  Selangor    RM2500 – RM4999   
7         Female         36-45     Malay  Selangor    RM5000 – RM6999   
8         Female         36-45     Malay  Selangor   RM7000 – RM10999   
9         Female         36-45     Malay        WP    RM2500 – RM4999   
10        Female         36-45     Malay        WP    RM5000 – RM6999   
11        Female         36-45     Malay        WP   RM7000 – RM10999   
12        Female  46 and above     Malay  Selangor 

In [22]:
total_rows = len(df)

# List of columns to process for purchase likelihood
purchase_columns = [col for col in df.columns if "Purchase" in col]

# Loop through each column to calculate likely and unlikely percentages
for column in purchase_columns:
    if column in df.columns:
        # Group and sum the purchase likelihood count per demographic group
        purchase_counts = df.groupby(['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income'])[column].sum().reset_index()

        # Rename the column for clarity
        purchase_counts.rename(columns={column: f'{column}_count'}, inplace=True)

        # Merge into pivot_table
        pivot_table = pivot_table.merge(purchase_counts, on=['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income'], how='left')

        # Fill NaNs with 0
        pivot_table[f'{column}_count'] = pivot_table[f'{column}_count'].fillna(0)

        # Calculate percentages
        pivot_table[f'{column} (Likely)'] = (pivot_table[f'{column}_count'] / total_rows) * 100
        pivot_table[f'{column} (Unlikely)'] = ((pivot_table['Response ID'] - pivot_table[f'{column}_count']) / total_rows) * 100

        # Round the results
        pivot_table[f'{column} (Likely)'] = pivot_table[f'{column} (Likely)']
        pivot_table[f'{column} (Unlikely)'] = pivot_table[f'{column} (Unlikely)']
    else:
        print(f"Column '{column}' is missing in df.")

# Drop intermediate count columns
intermediate_columns = [f'{col}_count' for col in purchase_columns if f'{col}_count' in pivot_table.columns]
pivot_table.drop(columns=intermediate_columns, inplace=True)

# Define display columns
display_columns = ['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income']
for column in purchase_columns:
    if f'{column} (Likely)' in pivot_table.columns and f'{column} (Unlikely)' in pivot_table.columns:
        display_columns.extend([f'{column} (Likely)', f'{column} (Unlikely)'])

print(pivot_table[display_columns])

    Gender     Age_range Ethnicity     State Avg_monthly_income  \
0   Female         26-35     Malay  Selangor    RM2500 – RM4999   
1   Female         26-35     Malay  Selangor    RM5000 – RM6999   
2   Female         26-35     Malay  Selangor   RM7000 – RM10999   
3   Female         26-35     Malay        WP    RM2500 – RM4999   
4   Female         26-35     Malay        WP    RM5000 – RM6999   
5   Female         26-35     Malay        WP   RM7000 – RM10999   
6   Female         36-45     Malay  Selangor    RM2500 – RM4999   
7   Female         36-45     Malay  Selangor    RM5000 – RM6999   
8   Female         36-45     Malay  Selangor   RM7000 – RM10999   
9   Female         36-45     Malay        WP    RM2500 – RM4999   
10  Female         36-45     Malay        WP    RM5000 – RM6999   
11  Female         36-45     Malay        WP   RM7000 – RM10999   
12  Female  46 and above     Malay  Selangor    RM2500 – RM4999   
13  Female  46 and above     Malay  Selangor    RM5000 – RM699

In [23]:
# total_rows = len(df)

# # List of columns to process for current usage
# purchase_columns = [col for col in df.columns if "Purchase" in col]

# # Loop through each column to calculate usage and non-usage percentages
# for column in purchase_columns:
#     if column in pivot_table.columns:
#         pivot_table[f'Not Purchase {column}'] = pivot_table['Response ID'] - pivot_table[column]
#         pivot_table[f'{column} (Likely)'] = (pivot_table[column] / total_rows) * 100
#         pivot_table[f'{column} (Unlikely)'] = (pivot_table[f'Not Purchase {column}'] / total_rows) * 100

#         # Round the percentages to 1 decimal place
#         pivot_table[f'{column} (Likely)'] = pivot_table[f'{column} (Likely)'].round(1)
#         pivot_table[f'{column} (Unlikely)'] = pivot_table[f'{column} (Unlikely)'].round(1)
#     else:
#         print(f"Column '{column}' is missing in pivt_table.")

# # Drop the original columns after processing
# pivot_table.drop(columns=[col for col in purchase_columns if col in pivot_table.columns], inplace=True)

# # Drop columns starting with "Not Purchase"
# not_using_columns = [col for col in pivot_table.columns if col.startswith('Not Purchase')]
# pivot_table.drop(columns=not_using_columns, inplace=True)

# # Define the columns for display, ensuring usage and non-usage are side by side
# display_columns = ['Gender', 'Age_range', 'Ethnicity','State','Avg_monthly_income']
# for column in purchase_columns:
#     if f'{column} (Likely)' in pivot_table.columns and f'{column} (Unlikely)' in pivot_table.columns:
#         display_columns.extend([f'{column} (Likely)', f'{column} (Unlikely)'])

# print(pivot_table[display_columns])

In [24]:
pivot_table.shape

(18, 83)

In [25]:
pivot_table = pivot_table.drop(columns=['Response ID'])
pivot_table.columns

Index(['Gender', 'Age_range', 'Ethnicity', 'State', 'Avg_monthly_income',
       'Clarins Purchase Likelihood', 'Clinique Purchase Likelihood',
       'Dior Purchase Likelihood', 'Estée Lauder Purchase Likelihood',
       'Innisfree Purchase Likelihood', 'Kiehl’s Purchase Likelihood',
       'Lancôme Purchase Likelihood', 'Laneige Purchase Likelihood',
       'L’Occitane Purchase Likelihood', 'SK-II Purchase Likelihood',
       'Shiseido Purchase Likelihood', 'Clarins Brand Awareness (%)',
       'Clarins Brand Unawareness (%)', 'Clinique Brand Awareness (%)',
       'Clinique Brand Unawareness (%)', 'Estée Lauder Brand Awareness (%)',
       'Estée Lauder Brand Unawareness (%)', 'Kiehl’s Brand Awareness (%)',
       'Kiehl’s Brand Unawareness (%)', 'L’Occitane Brand Awareness (%)',
       'L’Occitane Brand Unawareness (%)', 'Lancôme Brand Awareness (%)',
       'Lancôme Brand Unawareness (%)', 'Laneige Brand Awareness (%)',
       'Laneige Brand Unawareness (%)', 'Shiseido Brand Aware

In [26]:
merged_cleanser = pivot_table

merged_cleanser.to_csv('pivot-loki-cleanser.csv',float_format='%.1f', index=False)
merged_cleanser.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 82 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Gender                                       18 non-null     object 
 1   Age_range                                    18 non-null     object 
 2   Ethnicity                                    18 non-null     object 
 3   State                                        18 non-null     object 
 4   Avg_monthly_income                           18 non-null     object 
 5   Clarins Purchase Likelihood                  18 non-null     int64  
 6   Clinique Purchase Likelihood                 18 non-null     int64  
 7   Dior Purchase Likelihood                     18 non-null     int64  
 8   Estée Lauder Purchase Likelihood             18 non-null     int64  
 9   Innisfree Purchase Likelihood                18 non-null     int64  
 10  Kieh

### Preparing each df for each sheets in xlsx

In [27]:
columns_to_process1 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Current Usage (%)" in col]
columns_to_process1

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Clarins Current Usage (%)',
 'Clinique Current Usage (%)',
 'Estée Lauder Current Usage (%)',
 'Kiehl’s Current Usage (%)',
 'L’Occitane Current Usage (%)',
 'Lancôme Current Usage (%)',
 'Laneige Current Usage (%)',
 'Shiseido Current Usage (%)',
 'Innisfree Current Usage (%)',
 'SK-II Current Usage (%)',
 'Dior Current Usage (%)']

In [28]:
columns_to_process2 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Current Non-usage (%)" in col]
columns_to_process2

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Clarins Current Non-usage (%)',
 'Clinique Current Non-usage (%)',
 'Estée Lauder Current Non-usage (%)',
 'Kiehl’s Current Non-usage (%)',
 'L’Occitane Current Non-usage (%)',
 'Lancôme Current Non-usage (%)',
 'Laneige Current Non-usage (%)',
 'Shiseido Current Non-usage (%)',
 'Innisfree Current Non-usage (%)',
 'SK-II Current Non-usage (%)',
 'Dior Current Non-usage (%)']

In [29]:
columns_to_process3 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Purchase Likelihood (Likely)" in col]
columns_to_process3

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'SK-II Purchase Likelihood (Likely)',
 'Kiehl’s Purchase Likelihood (Likely)',
 'Innisfree Purchase Likelihood (Likely)',
 'Laneige Purchase Likelihood (Likely)',
 'Dior Purchase Likelihood (Likely)',
 'Clarins Purchase Likelihood (Likely)',
 'Estée Lauder Purchase Likelihood (Likely)',
 'Clinique Purchase Likelihood (Likely)',
 'L’Occitane Purchase Likelihood (Likely)',
 'Lancôme Purchase Likelihood (Likely)',
 'Shiseido Purchase Likelihood (Likely)']

In [30]:
columns_to_process4 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Purchase Likelihood (Unlikely)" in col]
columns_to_process4

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'SK-II Purchase Likelihood (Unlikely)',
 'Kiehl’s Purchase Likelihood (Unlikely)',
 'Innisfree Purchase Likelihood (Unlikely)',
 'Laneige Purchase Likelihood (Unlikely)',
 'Dior Purchase Likelihood (Unlikely)',
 'Clarins Purchase Likelihood (Unlikely)',
 'Estée Lauder Purchase Likelihood (Unlikely)',
 'Clinique Purchase Likelihood (Unlikely)',
 'L’Occitane Purchase Likelihood (Unlikely)',
 'Lancôme Purchase Likelihood (Unlikely)',
 'Shiseido Purchase Likelihood (Unlikely)']

In [31]:
columns_to_process5 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Awareness" in col]
columns_to_process5

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Clarins Brand Awareness (%)',
 'Clinique Brand Awareness (%)',
 'Estée Lauder Brand Awareness (%)',
 'Kiehl’s Brand Awareness (%)',
 'L’Occitane Brand Awareness (%)',
 'Lancôme Brand Awareness (%)',
 'Laneige Brand Awareness (%)',
 'Shiseido Brand Awareness (%)',
 'Innisfree Brand Awareness (%)',
 'SK-II Brand Awareness (%)',
 'Dior Brand Awareness (%)']

In [32]:
columns_to_process6 = merged_cleanser.columns[0:5].tolist() + [col for col in merged_cleanser.columns if "Unawareness" in col]
columns_to_process6

['Gender',
 'Age_range',
 'Ethnicity',
 'State',
 'Avg_monthly_income',
 'Clarins Brand Unawareness (%)',
 'Clinique Brand Unawareness (%)',
 'Estée Lauder Brand Unawareness (%)',
 'Kiehl’s Brand Unawareness (%)',
 'L’Occitane Brand Unawareness (%)',
 'Lancôme Brand Unawareness (%)',
 'Laneige Brand Unawareness (%)',
 'Shiseido Brand Unawareness (%)',
 'Innisfree Brand Unawareness (%)',
 'SK-II Brand Unawareness (%)',
 'Dior Brand Unawareness (%)']

In [33]:
# Create DataFrames for each set of columns
df1 = merged_cleanser[columns_to_process1]
df2 = merged_cleanser[columns_to_process2]
df3 = merged_cleanser[columns_to_process3]
df4 = merged_cleanser[columns_to_process4]
df5 = merged_cleanser[columns_to_process5]
df6 = merged_cleanser[columns_to_process6]

In [34]:
df1 = df1.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df1['Brand'] = df1['Brand'].str.replace(
    "Current Usage (%)", 
    "", regex=False
)

df1.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Dior,0.0
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Dior,0.0


In [35]:
df2 = df2.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df2['Brand'] = df2['Brand'].str.replace(
    "Current Non-usage (%)", 
    "", regex=False
)

df2.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Dior,0.983146
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Dior,0.280899


In [36]:
df3 = df3.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df3['Brand'] = df3['Brand'].str.replace(
    "Purchase Likelihood (Likely)", 
    "", regex=False
)

df3.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Shiseido,0.0
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Shiseido,0.0


In [37]:
df4 = df4.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df4['Brand'] = df4['Brand'].str.replace(
    "Purchase Likelihood (Unlikely)", 
    "", regex=False
)

df4.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Shiseido,0.983146
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Shiseido,0.280899


In [38]:
df5 = df5.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df5['Brand'] = df5['Brand'].str.replace(
    "Brand Awareness (%)", 
    "", regex=False
)

df5.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Dior,0.421348
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Dior,0.000000


In [39]:
df6 = df6.melt(id_vars=['Gender','Age_range','Ethnicity','State','Avg_monthly_income'], var_name='Brand', value_name='Percentage')

df6['Brand'] = df6['Brand'].str.replace(
    "Brand Unawareness (%)", 
    "", regex=False
)

df6.tail(2)

,Gender,Age_range,Ethnicity,State,Avg_monthly_income,Brand,Percentage
196,Female,46 and above,Malay,WP,RM5000 – RM6999,Dior,0.561798
197,Female,46 and above,Malay,WP,RM7000 – RM10999,Dior,0.280899


### Composite key creation and save into another excel file

In [40]:
# # Dictionary to store DataFrames with their respective attributes
# dataframes = {
#     'Current Usage': df1,
#     'Current Non-usage': df2,
#     'Purchase Likelihood (Likely)': df3,
#     'Purchase Likelihood (Unlikely)': df4,
#     'Awareness': df5,
#     'Unawareness': df6
# }

# def add_category_and_attribute(df, attribute_name):
#     df['Category'] = 'Cleaner'
#     df['Country'] = 'Malaysia'
#     df['Year'] = 2022
#     df['Attributes'] = attribute_name
#     return df

# # Process all DataFrames and filter out empty ones
# processed_dfs = [add_category_and_attribute(df, name) for name, df in dataframes.items() if df is not None and not df.empty]

# # Concatenate all DataFrames into a single DataFrame
# final_df = pd.concat(processed_dfs, ignore_index=True) if processed_dfs else None

# # Generate Composite Key after merging
# if final_df is not None:
#     # Ensure all columns in the composite key are strings
#     final_df['Composite_Key'] = (
#         final_df['Ethnicity'].astype(str) + '_' +
#         # final_df['Brand'].astype(str) + '_' +
#         final_df['Age_range'].astype(str) + '_' +
#         final_df['Gender'].astype(str) + '_' +
#         final_df['Avg_monthly_income'].astype(str) + '_' +
#         final_df['State'].astype(str)
#         # final_df['Category'].astype(str) + '_' +
#         # final_df['Country'].astype(str) + '_' +
#         # final_df['Year'].astype(str) + '_' +
#         # final_df['Attributes'].astype(str)
#     )

# print(final_df['Attributes'].value_counts())

In [41]:
# Dictionary to store DataFrames with their respective attributes
dataframes = {
    'Current Usage': df1,
    'Current Non-usage': df2,
    'Purchase Likelihood (Likely)': df3,
    'Purchase Likelihood (Unlikely)': df4,
    'Awareness': df5,
    'Unawareness': df6
}

def add_category_and_attribute(df, attribute_name):
    df['Category'] = 'Cleanser'
    df['Country'] = 'Malaysia'
    df['Year'] = 2020
    df['Attributes'] = attribute_name
    return df

# Process all DataFrames and filter out empty ones
processed_dfs = [add_category_and_attribute(df, name) for name, df in dataframes.items() if df is not None and not df.empty]

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(processed_dfs, ignore_index=True) if processed_dfs else None

# Generate Composite Key after merging
if final_df is not None:
    # Ensure all columns in the composite key are strings
    final_df['Composite_Key'] = (
        final_df['Ethnicity'].astype(str) + '_' +
        # final_df['Brand'].astype(str) + '_' +
        final_df['Age_range'].astype(str) + '_' +
        final_df['Gender'].astype(str) + '_' +
        final_df['Avg_monthly_income'].astype(str) + '_' +
        final_df['State'].astype(str)
        # final_df['Category'].astype(str) + '_' +
        # final_df['Country'].astype(str) + '_' +
        # final_df['Year'].astype(str) + '_' +
        # final_df['Attributes'].astype(str)
    )
    
    # Write to Excel
#     final_df.to_excel('cleanser-loki-final.xlsx', sheet_name='Cleaner', index=False)
#     print("Excel file created successfully!")
# else:
#     print("No valid DataFrames to write.")

# # Define the output directory
# output_dir = Path(r"C:\Users\aisar\OneDrive\Documents\Mobile-Data-App\BLS\CLEANED")
# output_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

# # Define the file path
# output_file = output_dir / "maxis-final.xlsx"

# # Write to Excel
# final_df.to_excel(output_file, sheet_name='Maxis', float_format='%.1f', index=False)
# print(f"Excel file created successfully: {output_file}")

In [42]:
# Remove rows where Percentage is 0
final_df = final_df[final_df['Percentage'] != 0]

# Write to Excel
final_df.to_excel('Cleanser-loki-final.xlsx', sheet_name='Cleanser', index=False)
print("Excel file created successfully!")

Excel file created successfully!


In [43]:
final_df['Percentage'] = pd.to_numeric(final_df['Percentage'], errors='coerce')
attribute_percentages = final_df.groupby('Attributes')['Percentage'].sum().reset_index()
attribute_percentages['Percentage'] = attribute_percentages['Percentage'].round(1)
print(attribute_percentages)

                       Attributes  Percentage
0                       Awareness       390.3
1               Current Non-usage       952.9
2                   Current Usage       147.1
3    Purchase Likelihood (Likely)        59.3
4  Purchase Likelihood (Unlikely)      1040.7
5                     Unawareness       709.7


In [44]:
import pandas as pd

# Ensure Percentage is numeric and handle NaNs
final_df['Percentage'] = pd.to_numeric(final_df['Percentage'], errors='coerce')

# Create pivot table with 0 for missing values
pivot = final_df.pivot_table(
    index='Attributes',
    columns='Brand',
    values='Percentage',
    aggfunc='sum',
    fill_value=0,  # Critical for matching Excel's behavior
    margins=True,
    margins_name='Grand Total'
).round(1)

# Reset index/columns for clarity
pivot = pivot.rename_axis('Row Labels', axis=0).rename_axis('Column Labels', axis=1)
print(pivot)

Column Labels                   Clarins   Clinique   Dior   Estée Lauder   \
Row Labels                                                                  
Awareness                           30.3       38.9   27.1           34.0   
Current Non-usage                   87.2       86.1   88.3           90.2   
Current Usage                       12.8       13.9   11.7            9.8   
Purchase Likelihood (Likely)         6.6        8.4    1.4            3.5   
Purchase Likelihood (Unlikely)      93.4       91.6   98.6           96.5   
Unawareness                         69.7       61.1   72.9           66.0   
Grand Total                        300.0      300.0  300.0          300.0   

Column Labels                   Innisfree   Kiehl’s   Lancôme   Laneige   \
Row Labels                                                                 
Awareness                             36.8      30.8      34.7      40.9   
Current Non-usage                     85.8      87.1      87.1      87.1   
Cu